<a href="https://colab.research.google.com/github/LaizaCostaa/previsao_preco_imoveis/blob/main/previsao_precos_imoveis_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


** ANÁLISE DE PREÇO DE IMÓVEIS PARTE II **

Uma análise preditiva dos preços das casas, cuja análise exploratória está neste link: https://colab.research.google.com/drive/1UhBuudCS7fvnb5IUFOa4TLcMv-JuPMGe?usp=sharing


Decidi dividir em duas partes porque queria fazer uma análise exploratória mais detalhada e não deixar o código tão poluído com análise e aplicação do modelo ao mesmo tempo.


* Artigo da análise exploratória: https://medium.com/@laizamare/an%C3%A1lise-de-oportunidades-e-estrat%C3%A9gias-de-neg%C3%B3cio-no-setor-imobili%C3%A1rio-estudo-de-caso-house-d629e2378e78


* Artigo com análise preditiva: https://medium.com/@laizamare/prevendo-pre%C3%A7os-de-im%C3%B3veis-usando-random-forest-regressor-3793a4ecf995


In [25]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno


In [26]:

# ler o csv
df = pd.read_csv("/content/df.csv")
display(df)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,renovated,price group,month,year,location
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,98178,47.5112,-122.257,1340,5650,0,Group 1,10,2014,Seattle
1,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,98178,47.5112,-122.257,1340,5650,0,Group 1,10,2014,Tukwila
2,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,0,...,98178,47.5009,-122.248,1190,6780,0,Group 1,6,2014,Seattle
3,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,0,...,98178,47.5009,-122.248,1190,6780,0,Group 1,6,2014,Tukwila
4,4058801670,2014-07-17,445000.0,3,2.25,2100,8201,1.0,0,2,...,98178,47.5091,-122.244,2660,8712,0,Group 2,7,2014,Seattle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32610,2525049086,2014-10-03,2720000.0,4,3.25,3990,18115,2.0,0,0,...,98039,47.6177,-122.229,3450,16087,0,Group 4,10,2014,Medina
32611,2525049113,2014-07-25,1950000.0,4,3.50,4065,18713,2.0,0,0,...,98039,47.6209,-122.237,3070,18713,0,Group 4,7,2014,Medina
32612,3262300485,2015-04-21,2250000.0,5,5.25,3410,8118,2.0,0,0,...,98039,47.6295,-122.236,3410,16236,0,Group 4,4,2015,Medina
32613,6447300365,2014-11-13,2900000.0,5,4.00,5190,14600,2.0,0,1,...,98039,47.6102,-122.225,3840,19250,0,Group 4,11,2014,Medina


**PRÉ-PROCESSAMENTO DOS DADOS**

In [27]:
df_pre_tratado = df.drop(['date', 'price group', 'month', 'year', 'yr_built', 'yr_renovated', 'renovated', 'waterfront', 'condition', 'floors', 'view'], axis=1)


In [49]:
df_pre_tratado

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,grade,sqft_above,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,location
0,7129300520,221900.0,3,1.00,1180,5650,7,1180,0,98178,47.5112,-122.257,1340,5650,Seattle
1,7129300520,221900.0,3,1.00,1180,5650,7,1180,0,98178,47.5112,-122.257,1340,5650,Tukwila
2,4060000240,205425.0,2,1.00,880,6780,6,880,0,98178,47.5009,-122.248,1190,6780,Seattle
3,4060000240,205425.0,2,1.00,880,6780,6,880,0,98178,47.5009,-122.248,1190,6780,Tukwila
4,4058801670,445000.0,3,2.25,2100,8201,8,1620,480,98178,47.5091,-122.244,2660,8712,Seattle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32610,2525049086,2720000.0,4,3.25,3990,18115,11,3990,0,98039,47.6177,-122.229,3450,16087,Medina
32611,2525049113,1950000.0,4,3.50,4065,18713,10,4065,0,98039,47.6209,-122.237,3070,18713,Medina
32612,3262300485,2250000.0,5,5.25,3410,8118,11,3410,0,98039,47.6295,-122.236,3410,16236,Medina
32613,6447300365,2900000.0,5,4.00,5190,14600,11,5190,0,98039,47.6102,-122.225,3840,19250,Medina


In [28]:
one_hot = pd.get_dummies(df_pre_tratado['location'])

In [50]:
one_hot

,Algona,Auburn,Beaux Arts Village,Bellevue,Black Diamond,Bothell,Burien,Carnation,Clyde Hill,Covington,...,Renton,Sammamish,Seatac,Seattle,Shoreline,Snoqualmie,Tukwila,Vashon,Woodinville,Yarrow Point
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32613,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
df_tratado = df_pre_tratado.drop("location", axis=1)
df_tratado = df_tratado.join(one_hot)
df_tratado


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,grade,sqft_above,sqft_basement,zipcode,...,Renton,Sammamish,Seatac,Seattle,Shoreline,Snoqualmie,Tukwila,Vashon,Woodinville,Yarrow Point
0,7129300520,221900.0,3,1.00,1180,5650,7,1180,0,98178,...,0,0,0,1,0,0,0,0,0,0
1,7129300520,221900.0,3,1.00,1180,5650,7,1180,0,98178,...,0,0,0,0,0,0,1,0,0,0
2,4060000240,205425.0,2,1.00,880,6780,6,880,0,98178,...,0,0,0,1,0,0,0,0,0,0
3,4060000240,205425.0,2,1.00,880,6780,6,880,0,98178,...,0,0,0,0,0,0,1,0,0,0
4,4058801670,445000.0,3,2.25,2100,8201,8,1620,480,98178,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32610,2525049086,2720000.0,4,3.25,3990,18115,11,3990,0,98039,...,0,0,0,0,0,0,0,0,0,0
32611,2525049113,1950000.0,4,3.50,4065,18713,10,4065,0,98039,...,0,0,0,0,0,0,0,0,0,0
32612,3262300485,2250000.0,5,5.25,3410,8118,11,3410,0,98039,...,0,0,0,0,0,0,0,0,0,0
32613,6447300365,2900000.0,5,4.00,5190,14600,11,5190,0,98039,...,0,0,0,0,0,0,0,0,0,0


**DEFININDO DADOS DE TREINO E DADOS DE TESTE**

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
# definindo as variáveis
y = df_tratado['price']
x = df_tratado.loc[:, df_tratado.columns != "price"]

In [33]:
x

,id,bedrooms,bathrooms,sqft_living,sqft_lot,grade,sqft_above,sqft_basement,zipcode,lat,...,Renton,Sammamish,Seatac,Seattle,Shoreline,Snoqualmie,Tukwila,Vashon,Woodinville,Yarrow Point
0,7129300520,3,1.00,1180,5650,7,1180,0,98178,47.5112,...,0,0,0,1,0,0,0,0,0,0
1,7129300520,3,1.00,1180,5650,7,1180,0,98178,47.5112,...,0,0,0,0,0,0,1,0,0,0
2,4060000240,2,1.00,880,6780,6,880,0,98178,47.5009,...,0,0,0,1,0,0,0,0,0,0
3,4060000240,2,1.00,880,6780,6,880,0,98178,47.5009,...,0,0,0,0,0,0,1,0,0,0
4,4058801670,3,2.25,2100,8201,8,1620,480,98178,47.5091,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32610,2525049086,4,3.25,3990,18115,11,3990,0,98039,47.6177,...,0,0,0,0,0,0,0,0,0,0
32611,2525049113,4,3.50,4065,18713,10,4065,0,98039,47.6209,...,0,0,0,0,0,0,0,0,0,0
32612,3262300485,5,5.25,3410,8118,11,3410,0,98039,47.6295,...,0,0,0,0,0,0,0,0,0,0
32613,6447300365,5,4.00,5190,14600,11,5190,0,98039,47.6102,...,0,0,0,0,0,0,0,0,0,0


In [34]:
y

0         221900.0
1         221900.0
2         205425.0
3         205425.0
4         445000.0
           ...    
32610    2720000.0
32611    1950000.0
32612    2250000.0
32613    2900000.0
32614    1865000.0
Name: price, Length: 32615, dtype: float64

In [35]:
# Separando dados treino e teste

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [36]:
x_train

,id,bedrooms,bathrooms,sqft_living,sqft_lot,grade,sqft_above,sqft_basement,zipcode,lat,...,Renton,Sammamish,Seatac,Seattle,Shoreline,Snoqualmie,Tukwila,Vashon,Woodinville,Yarrow Point
14917,7680400140,3,3.25,3740,136915,11,3100,640,98166,47.4549,...,0,0,0,1,0,0,0,0,0,0
19051,3304300210,4,2.75,2700,7992,9,2700,0,98059,47.4831,...,1,0,0,0,0,0,0,0,0,0
12934,4463400195,2,1.00,1280,21750,6,1280,0,98001,47.3096,...,0,0,0,0,0,0,0,0,0,0
15120,2105200010,4,2.50,2030,39049,7,1530,500,98166,47.4413,...,0,0,0,0,0,0,0,0,0,0
6401,3735900205,4,2.50,2450,4080,8,1490,960,98115,47.6899,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10065,3375800220,3,2.50,2550,6021,7,2550,0,98030,47.3828,...,0,0,0,0,0,0,0,0,0,0
13535,629650380,4,2.50,1660,6724,7,1660,0,98001,47.2590,...,0,0,0,0,0,0,0,0,0,0
10391,5332200515,3,1.75,2650,5512,9,2250,400,98112,47.6265,...,0,0,0,1,0,0,0,0,0,0
20449,8146300015,4,2.50,2170,8680,7,1220,950,98004,47.6078,...,0,0,0,0,0,0,0,0,0,0


In [39]:
y_train

14917     710000.0
19051     572000.0
12934     170000.0
15120     515000.0
6401      793000.0
           ...    
10065     353000.0
13535     255000.0
10391    1050000.0
20449     685000.0
23965     972000.0
Name: price, Length: 22830, dtype: float64

**APLICAÇÃO DO MODELO**

In [40]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [41]:
# imputer para preencher os valores ausentes com a média
imputer = SimpleImputer(strategy='mean')

x_train_imputed = imputer.fit_transform(x_train)

In [42]:
#trazendo o modelo
rf_reg = RandomForestRegressor()
rf_reg.fit(x_train_imputed, y_train)

RandomForestRegressor()

In [43]:
RandomForestRegressor()

RandomForestRegressor()

In [44]:
#verificando os parâmetros de determinado imóvel nos dados de teste

y_test.iloc[0]

394000.0

In [51]:
x_test.iloc[0].head(10)

id               7.853341e+09
bedrooms         2.000000e+00
bathrooms        2.000000e+00
sqft_living      1.750000e+03
sqft_lot         2.731000e+03
grade            8.000000e+00
sqft_above       1.750000e+03
sqft_basement    0.000000e+00
zipcode          9.806500e+04
lat              4.751690e+01
Name: 26559, dtype: float64

In [46]:
# Verificando o valor sugerido pelo modelo considerando os parâmetros dos dados de teste

rf_reg.predict(x_test.iloc[0].values.reshape(1, -1))

array([382389.86])

* Valor bem próximo do encontrado nos dados de teste

In [47]:
# Verificando parâmetros de imóveis específicos


n = 1

print(y_test.iloc[n])
print(x_test.iloc[n].head(10))

rf_reg.predict(x_test.iloc[n].values.reshape(1, -1))

475000.0
id               2.997800e+09
bedrooms         3.000000e+00
bathrooms        2.500000e+00
sqft_living      1.310000e+03
sqft_lot         1.294000e+03
grade            8.000000e+00
sqft_above       1.180000e+03
sqft_basement    1.300000e+02
zipcode          9.811600e+04
lat              4.757730e+01
Name: 23138, dtype: float64


array([477461.14])



*  O Imóvel encontrado está no valore de 475.000 dólares e foi avaliado pelo modelo em 477.461, ou seja, muito próximo do valor real.
*   O imóvel atual tem menos quartos  do que o anterior, uma correlação que se alinha muito bem com a nossa análise exploratória.

**AVALIAÇÃO**

In [48]:
rf_reg.score(x_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


0.9135359509032139

* O modelo atingiu uma acurácia 91.35%